In [1]:

%load_ext sql

import pandas as pd
import sqlite3
%sql sqlite:///Course_DW.db 

'Connected: @Course_DW.db'

In [2]:
%%sql
ATTACH DATABASE 'CourseData.db' AS cd

 * sqlite:///Course_DW.db
Done.


[]

In [3]:
%%sql
SELECT *
FROM cd.PROGRAM
LIMIT 5

 * sqlite:///Course_DW.db
Done.


ProgramID,ProgramCode,ProgramName
1,None,None
2,AC,Accounting
3,AE,Applied Ethics
4,AH,Art History
5,AN,Asian Studies


In [4]:
%%sql

DROP TABLE IF EXISTS LOCATION1;
CREATE TABLE LOCATION1 (
    LocationID INTEGER PRIMARY KEY,
    Location_Code TEXT,
    Building TEXT,
    Room TEXT
);

DROP TABLE IF EXISTS INSTRUCTOR1;
CREATE TABLE INSTRUCTOR1 (
    InstructorID INTEGER PRIMARY KEY,
    InstructorName TEXT 
);

DROP TABLE IF EXISTS PROGRAM1;
CREATE TABLE PROGRAM1 (
    ProgramID INTEGER PRIMARY KEY,
    ProgramCode TEXT,
    ProgramName TEXT
);

DROP TABLE IF EXISTS COURSE1;
CREATE TABLE COURSE1 (
    CatalogID TEXT,
    CourseID INTEGER PRIMARY KEY,
    ProgramID INTEGER NOT NULL,
    CourseTitle TEXT NOT NULL,
    Description TEXT,
    Attributes TEXT,
    Coreqs TEXT,
    Prereqs TEXT,
    Fees TEXT,
    Credits INTEGER NOT NULL,
    FOREIGN KEY (ProgramID) REFERENCES PROGRAM (ProgramID)
);

DROP TABLE IF EXISTS COURSE_MEETING1;
CREATE TABLE COURSE_MEETING1 (
    CourseMeetingID INTEGER PRIMARY KEY,
    CatalogID TEXT,
    Location TEXT,
    StartDateTime TEXT,
    EndDateTime TEXT,
    FOREIGN KEY (Location) REFERENCES LOCATION (Location)
);

 * sqlite:///Course_DW.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [5]:
%%sql
DELETE FROM LOCATION1;
INSERT INTO LOCATION1 (Location_Code, Building, Room)
SELECT Location_Code, Building, Room
FROM cd.LOCATION;

 * sqlite:///Course_DW.db
0 rows affected.
207 rows affected.


[]

In [6]:
%%sql
DELETE FROM INSTRUCTOR1;

INSERT INTO INSTRUCTOR1 (InstructorName)
SELECT DISTINCT InstructorName
FROM cd.INSTRUCTOR;

 * sqlite:///Course_DW.db
0 rows affected.
1048 rows affected.


[]

In [7]:
%%sql
DELETE FROM PROGRAM1;

INSERT INTO PROGRAM1  (ProgramCode, ProgramName)
SELECT DISTINCT ProgramCode, ProgramName
FROM cd.PROGRAM;

 * sqlite:///Course_DW.db
0 rows affected.
81 rows affected.


[]

In [8]:
%%sql
DELETE FROM COURSE1;

INSERT INTO COURSE1 (CatalogID, ProgramID, CourseTitle, Description, Attributes, Coreqs, Prereqs, Fees, Credits)
SELECT DISTINCT CatalogID, COURSE.ProgramID, CourseTitle, Description, Attributes, Coreqs, Prereqs, Fees, Credits
FROM cd.COURSE

 * sqlite:///Course_DW.db
0 rows affected.
1972 rows affected.


[]

In [9]:
%%sql
DELETE FROM COURSE_MEETING1;

INSERT INTO COURSE_MEETING1 (CatalogID, Location, StartDateTime, EndDateTime)
SELECT DISTINCT CatalogID, Location, StartDateTime, EndDateTime
FROM cd.COURSE_MEETING

 * sqlite:///Course_DW.db
0 rows affected.
280529 rows affected.


[]

In [10]:
%%sql
--FACT TABLE

DROP TABLE IF EXISTS COURSE_FACTS;
CREATE TABLE COURSE_FACTS (
    CourseOfferingID INTEGER PRIMARY KEY,
    CatalogID TEXT NOT NULL,
    CourseTitle TEXT NOT NULL,
    InstructorName TEXT NOT NULL,
    Section TEXT, 
    Meetings TEXT NOT NULL,
    Timecodes TEXT NOT NULL,
    Term TEXT NOT NULL,
    StartDay TEXT NOT NULL,
    End TEXT NOT NULL,    
    Cap INTEGER NOT NULL,
    Act INTEGER NOT NULL,
    Rem INTEGER NOT NULL,
    FOREIGN KEY (CatalogID) REFERENCES COURSE_MEETING1 (CatalogID)
    FOREIGN KEY (InstructorName) REFERENCES INSTRUCTOR1 
    (InstructorName)
    FOREIGN KEY (CourseTitle) REFERENCES COURSE1 (CourseTitle)
);



 * sqlite:///Course_DW.db
Done.
Done.


[]

In [ ]:
%%sql
-- DELETE FROM COURSE_FACTS;

-- INSERT INTO COURSE_FACTS (CatalogID, CourseTitle, InstructorName, Section, Meetings, TimeCodes, Term, StartDay, End, Cap, Act, Rem)
-- SELECT DISTINCT COURSE1.CatalogID, COURSE1.CourseTitle, InstructorName, Section, Meetings, TimeCodes, Term, StartDay, End, Cap, Act, Rem
-- FROM cd.COURSE_OFFERING
     -- JOIN COURSE_MEETING1 ON (cd.COURSE_MEETING1.CatalogID = cd.CatalogID)
     -- JOIN INSTRUCTOR1 ON (cd.INSTRUCTOR1.InstructorName = cd.InstructorName)
     -- LEFT JOIN COURSE1 ON (COURSE1.CatalogID = cd.COURSE.CatalogID)

In [16]:
%%sql
DELETE FROM COURSE_FACTS;

INSERT INTO COURSE_FACTS (CatalogID, CourseTitle, InstructorName, Section, Meetings, TimeCodes, Term, StartDay, End, Cap, Act, Rem)
SELECT DISTINCT CatalogID, CourseTitle, InstructorName, Section, Meetings, TimeCodes, Term, StartDay, End, Cap, Act, Rem
FROM cd.COURSE_OFFERING
        JOIN COURSE_MEETING1 USING (CatalogID)
     -- JOIN INSTRUCTOR1 ON (cd.INSTRUCTOR1.InstructorName = cd.InstructorName)
     -- LEFT JOIN COURSE1 ON (COURSE1.CatalogID = cd.COURSE.CatalogID)

 * sqlite:///Course_DW.db
0 rows affected.
283781 rows affected.


[]

In [29]:
%%sql
DELETE FROM COURSE_FACTS;
INSERT INTO COURSE_FACTS (CatalogID, CourseTitle, InstructorName, Section, Meetings, TimeCodes, Term, StartDay, End, Cap, Act, Rem)
    SELECT DISTINCT cd.COURSE_MEETING.CatalogID, cd.COURSE.CourseTitle, cd.INSTRUCTOR.InstructorName, Section, Meetings, TimeCodes, Term, StartDay, End, Cap, Act, Rem
FROM cd.COURSE_OFFERING
        JOIN COURSE_MEETING1 USING (CatalogID)
        JOIN INSTRUCTOR1 USING (InstructorName)
        JOIN COURSE1 ON (COURSE1.CatalogID = cd.COURSE.CatalogID)

 * sqlite:///Course_DW.db
0 rows affected.
(sqlite3.OperationalError) no such column: cd.COURSE_MEETING.CatalogID
[SQL: INSERT INTO COURSE_FACTS (CatalogID, CourseTitle, InstructorName, Section, Meetings, TimeCodes, Term, StartDay, End, Cap, Act, Rem)
    SELECT DISTINCT cd.COURSE_MEETING.CatalogID, cd.COURSE.CourseTitle, cd.INSTRUCTOR.InstructorName, Section, Meetings, TimeCodes, Term, StartDay, End, Cap, Act, Rem
FROM cd.COURSE_OFFERING
        JOIN COURSE_MEETING1 USING (CatalogID)
        JOIN INSTRUCTOR1 USING (InstructorName)
        JOIN COURSE1 ON (COURSE1.CatalogID = cd.COURSE.CatalogID)]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [18]:
%%sql

SELECT *
FROM COURSE_FACTS
LIMIT 5;


 * sqlite:///Course_DW.db
Done.


CourseOfferingID,CatalogID,CourseTitle,InstructorName,Section,Meetings,Timecodes,Term,StartDay,End,Cap,Act,Rem
1,AC 0011,Introduction to Financial Accounting,Michael P. Coyne,C01,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0800am-0915am 09/02-12/08 DSB 105'],Fall2014,2014-09-02T08:00:00,2014-09-02T09:15:00,0,31,-31
2,AC 0011,Introduction to Financial Accounting,Michael P. Coyne,C01,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0800am-0915am 09/02-12/08 DSB 105'],Fall2014,2014-09-05T08:00:00,2014-09-05T09:15:00,0,31,-31
3,AC 0011,Introduction to Financial Accounting,Michael P. Coyne,C01,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0800am-0915am 09/02-12/08 DSB 105'],Fall2014,2014-09-09T08:00:00,2014-09-09T09:15:00,0,31,-31
4,AC 0011,Introduction to Financial Accounting,Michael P. Coyne,C01,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0800am-0915am 09/02-12/08 DSB 105'],Fall2014,2014-09-12T08:00:00,2014-09-12T09:15:00,0,31,-31
5,AC 0011,Introduction to Financial Accounting,Michael P. Coyne,C01,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0800am-0915am 09/02-12/08 DSB 105'],Fall2014,2014-09-16T08:00:00,2014-09-16T09:15:00,0,31,-31


In [37]:
%%sql

SELECT 
    (SELECT Count(*) FROM LOCATION1) as Locations,
    (SELECT Count(*) FROM INSTRUCTOR1) as Instructors,
    (SELECT Count(*) FROM PROGRAM1) as Programs,
    (SELECT Count(*) FROM COURSE1) as Courses,
    (SELECT Count(*) FROM COURSE_MEETING1) as Course_Meetings
    ;

 * sqlite:///Course_DW.db
Done.


Locations,Instructors,Programs,Courses,Course_Meetings
207,1048,81,1972,280529


In [38]:
%%sql
vacuum;

 * sqlite:///Course_DW.db
Done.


[]